## threading 的多线程并发
- 对比多进程并发： 
  - 优点 ： 资源消耗少
  - 缺点 ： 需要注意共享该资源的争夺  (注意GIL问题)


In [ ]:
# thread_server
from socket import *
from threading import *
HOST = '0.0.0.0'
POST = 8888
ADDR = (HOST,POST)

s = socket()
s.bind(ADDR)
s.listen(5)

while True:
    try:
        connfd,adr = s.accept()
    except KeyboardInterrupt:
        s.close()
        sys.exit('服务器退出')
    except Exception as e:
        print(e)
        continue
        
    t = Thread
        